In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse

In [14]:
def get_event_results(title, date, place, online_table_link):
    if online_table_link is None:
        return None
    df = pd.DataFrame(columns=['title','date','place','online_link','category','rank', 'name', 'club', 'points'])
    page = requests.get(online_table_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    all_res = soup.find_all('a', {'href': re.compile(r'CAT[0-9]{3}RS')})
    qty = 0
    for link in all_res:
        page = requests.get(urllib.parse.urljoin(online_table_link, link.get('href')))
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            if len(soup.find_all('h2')) < 2:
                continue
            category = soup.find_all('h2')[1].text
            lines = soup.find_all('tr')
            for line in lines:
                cells = line.find_all('td')
                if len(cells) >= 4:
                    rank = cells[0].text
                    name = cells[1].a.text
                    club = cells[1].text[len(name):]
                    name = ' '.join([x.strip() for x in name.split()])
                    name = name.replace("ё", "е")
                    points = cells[2].text
                    df.loc[len(df)] = [title, date, place, online_link, category, rank, name, club, points]
                    
    return df

In [15]:
df = pd.DataFrame(columns=['title','date','place','online_link','category','rank', 'name', 'club', 'points'])

for season in range(2020, 2025):
    url = f"http://ffkkmo.ru/calendar/?season={season}"
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    for competition in soup.find_all('tr'):
        title = competition.find_all('td')[1]
        if title.a is not None:
            online_link = None
            proto_link = None
            event_link = title.a.get('href')
            event_page = requests.get(event_link)
            event_soup = BeautifulSoup(event_page.text, 'html.parser')
            title = event_soup.find("h1", "entry-title").text
            date = event_soup.find("div", "competition-date").p.text.split('-')[0].strip()
            print(date)
            place = event_soup.find("div", "competition-place").p.span.text.strip()
            online = event_soup.find("div", "competition-file")
            if online is not None:
                online_link = online.a.get('href')
                online_link = online_link if online_link[-1] == '/' else online_link + '/'
            proto_el = event_soup.find("span", "glyphicon glyphicon-file")
            if proto_el is not None:
                proto_link = proto_el.parent.get('href')        
            df = pd.concat([df, get_event_results(title, date, place, online_link)])
            

27.09.2020
03.10.2020
17.10.2020
24.10.2020
31.10.2020
06.11.2020
06.11.2020
14.11.2020
16.11.2020
28.11.2020
01.12.2020
04.12.2020
12.12.2020
15.12.2020
17.12.2020
26.12.2020
29.12.2020
09.01.2021
12.01.2021
16.01.2021
02.02.2021
09.02.2021
12.02.2021
20.02.2021
27.02.2021
02.03.2021
13.03.2021
20.03.2021
27.03.2021
30.03.2021
03.04.2021
09.04.2021
17.04.2021
20.04.2021
24.04.2021
27.04.2021
01.05.2021
08.05.2021
15.05.2021
29.05.2021
02.06.2021
12.06.2021
18.09.2021
25.09.2021
02.10.2021
09.10.2021
16.10.2021
23.10.2021
30.10.2021
13.11.2021
20.11.2021
27.11.2021
04.12.2021
07.12.2021
11.12.2021
14.12.2021
17.12.2021
25.12.2021
28.12.2021
15.01.2022
22.01.2022
26.01.2022
29.01.2022
05.02.2022
12.02.2022
13.02.2022
19.02.2022
26.02.2022
01.03.2022
05.03.2022
12.03.2022
15.03.2022
19.03.2022
26.03.2022
29.03.2022
02.04.2022
09.04.2022
10.04.2022
12.04.2022
15.04.2022
23.04.2022
26.04.2022
30.04.2022
07.05.2022
10.05.2022
14.05.2022
18.05.2022
21.05.2022
24.05.2022
28.05.2022
04.06.2022

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24717 entries, 0 to 37
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        24717 non-null  object
 1   date         24717 non-null  object
 2   place        24717 non-null  object
 3   online_link  24717 non-null  object
 4   category     24717 non-null  object
 5   rank         24717 non-null  object
 6   name         24717 non-null  object
 7   club         24717 non-null  object
 8   points       24717 non-null  object
dtypes: object(9)
memory usage: 1.9+ MB


In [17]:
df.to_csv("ffkkmo_dump.csv")